In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv("project_metrics_v3.csv")

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sns.jointplot(x="STA-1", y="POP-4", data=df, alpha=0.5)

In [ ]:
X = df['STA-1']
y = df['POP-4']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=42)

X_train

In [ ]:
sns.lmplot(x = 'TEC-1',
           y = 'POP-4',
           data = df
           )